In [ ]:
onnx_model_path = "./model/model.onnx"

ExecutionProvider="QNNExecutionProvider"

if ExecutionProvider == "VitisAIExecutionProvider":
    provider_options={
            "target": "RyzenAI_google_bert",
            "cache_dir": "C:\\temp",
            "dd_write_debug_files": "true",
            "log_level": "info"
        }
else:
    # default for QNNExecutionProvider
    provider_options={"backend_path": "QnnHtp.dll"}


In [ ]:
inputs = "This is an example sentence."

In [ ]:
import numpy as np
import onnxruntime
import torch
import torch.nn.functional as F
from pathlib import Path
from transformers import AutoModel, AutoTokenizer

In [ ]:
def mean_pooling(model_output, attention_mask):
    token_embeddings = torch.tensor(model_output[0])
    input_mask_expanded = attention_mask.unsqueeze(-1).expand_as(token_embeddings).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('google-bert/bert-base-multilingual-cased')
encoded_input = tokenizer(
    inputs,
    padding="max_length",
    max_length=128,
    truncation=True,
    add_special_tokens=True,
    return_tensors="pt",
)

In [ ]:
# get text embedding from quantized model.
options = onnxruntime.SessionOptions()
# options.add_session_config_entry("session.disable_cpu_ep_fallback", "1")
session = onnxruntime.InferenceSession(onnx_model_path,
    sess_options=options,
    providers=[ExecutionProvider],
    provider_options=[provider_options])

input_ids = encoded_input["input_ids"]
attention_mask = encoded_input["attention_mask"]
token_type_ids = encoded_input["token_type_ids"]
inputs = {
    "input_ids": input_ids.long().cpu().numpy(),
    "attention_mask": attention_mask.long().cpu().numpy(),
    "token_type_ids": token_type_ids.long().cpu().numpy()
}

outputs = session.run(None, inputs)
embeds_1 = mean_pooling(outputs, encoded_input['attention_mask'])
embeds_1 = F.normalize(embeds_1, p=2, dim=1)

In [ ]:
# get text embedding from orinal model, as ground truth.
model = AutoModel.from_pretrained('google-bert/bert-base-multilingual-cased').eval()
with torch.no_grad():
    outputs = model(**encoded_input)
    embeds_2 = mean_pooling(outputs, encoded_input['attention_mask'])
    embeds_2 = F.normalize(embeds_2, p=2, dim=1)

In [ ]:
similarity = F.cosine_similarity(embeds_1, embeds_2).item()
print("Similarity: ", similarity)